In [7]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from pathlib import Path
from scipy.optimize import lsq_linear
from rw_data_processing import *
from Data_synthesize import *
from tqdm import tqdm

plt.style.use("./rw_visualization.mplstyle")


In [8]:
import sys
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

# Color
current_palette = sns.color_palette()
current_palette


In [10]:
save_p_state = True # Set False if you do not want to save any parameters

# 1. Demographic parameters

## 1.1 Household

In [11]:
demographic_data_path = Path('./data/demographic_data')
family_size_data = pd.read_excel(demographic_data_path/'縣市戶數結構.xls', sheet_name='110')
family_size_data.columns = family_size_data.iloc[1]
family_size_data = family_size_data[3:23+1]
family_size_data = family_size_data[family_size_data['區  域  別'] != '臺  灣  省']
family_size_data = family_size_data[family_size_data['區  域  別'] != '福  建  省']
family_size_data = family_size_data[family_size_data['區  域  別'] != '澎  湖  縣']


In [12]:
# Load total number of famaliy each city
family_number_data = pd.read_excel(demographic_data_path/'縣市村里鄰戶數及人口數-110年.xls')
family_number_data.columns = family_number_data.iloc[1]
family_number_data = family_number_data[4:28]
family_number_data = family_number_data[family_number_data['區域別'] != '臺灣省']
family_number_data = family_number_data[family_number_data['區域別'] != '福建省']
family_number_data = family_number_data[family_number_data['區域別'] != '澎湖縣']
family_number_data = family_number_data[family_number_data['區域別'] != '連江縣']
family_number_data = family_number_data[family_number_data['區域別'] != '金門縣']
family_number = family_number_data['人口數'].to_numpy()


In [13]:
# Since the household cases are too less compare to Cheng et al. even when the household contact p close to 1. I thus adjust this probability.
family_size_dict = {}
for i, city in enumerate(family_size_data['區  域  別']):
    family_size_array = family_size_data[family_size_data['區  域  別'] == city].to_numpy()[0][2:-1]
    
    # Correct family size by family_number
    people_higher_than_6 = family_number[i] - np.sum(family_size_array[0:-1]*np.arange(1, 6))
    higher_than_6_array = np.array([family_size_array[-1]])
    
    b = np.array([people_higher_than_6, family_size_array[-1]])
    A = np.array([[j, 1] for j in range(6, 13)])  # Changed to include sizes 6 to 12
    
    res = lsq_linear(A.T, b, bounds=(0, 10**10), lsmr_tol='auto', verbose=0)
    family_size_array_tmp = np.append(family_size_array[0:-1], np.array(res.x))
    
    family_size_array = family_size_array_tmp
    family_size_dict[city.replace(' ', '')] = np.array(family_size_array/sum(family_size_array), dtype=float)
    
    plt.figure()
    plt.bar(np.arange(len(family_size_array)), family_size_array)
    plt.xticks(np.arange(len(family_size_array)), np.arange(len(family_size_array))+1)
    plt.xlabel('Household family size')
    plt.ylabel('Amount of household')
    plt.close()  # Comment out this line if you want to see the plots

## 1.2 School

In [14]:
school_p = {}

### 1.2.1 Elementary school

In [15]:
elementary_school_data = pd.read_excel(demographic_data_path/'國民小學校別資料.xls')
elementary_school_data.columns = elementary_school_data.iloc[1]
elementary_school_data = elementary_school_data[2::]
elementary_school_data = elementary_school_data[elementary_school_data['縣市名稱'] != '金門縣']
elementary_school_data = elementary_school_data[elementary_school_data['縣市名稱'] != '連江縣']
elementary_school_data = elementary_school_data[elementary_school_data['縣市名稱'] != '澎湖縣']


In [16]:
class_index = elementary_school_data.columns[6:11+1]
class_index_english = ['1st-grade', '2nd-grade',
                       '3rd-grade', '4th-grade', '5th-grade', '6th-grade']
student_number_index = elementary_school_data.columns[15:26+1]
for i in range(len(class_index)):
    plt.figure(figsize=(12, 4))
    # Remove empty schools
    elementary_school_data_temp = elementary_school_data[elementary_school_data[class_index[i]] != 0]
    student_per_class = (elementary_school_data_temp[student_number_index[2*i]] +
                         elementary_school_data_temp[student_number_index[2*i+1]])/elementary_school_data_temp[class_index[i]]
    n = plt.hist(student_per_class, bins=range(int(max(student_per_class))))
    probability = np.zeros(int(n[1].max()) + 1)
    probability[n[1][0:-1].astype(int)] = n[0]/sum(n[0])
    school_p[i+7] = probability

    plt.xticks(np.arange(0, int(max(student_per_class)), 5))
    plt.xlabel('Number of ' + class_index_english[i] + ' students per class')
    plt.ylabel('Number of school')
    plt.close()


### 1.2.2 Junior high school

In [17]:
junior_high_school_data = pd.read_excel(demographic_data_path/'國民中學校別資料.xlsx')
junior_high_school_data.columns = junior_high_school_data.iloc[1]
junior_high_school_data = junior_high_school_data[2::]
junior_high_school_data = junior_high_school_data[junior_high_school_data['縣市名稱'] != '金門縣']
junior_high_school_data = junior_high_school_data[junior_high_school_data['縣市名稱'] != '連江縣']
junior_high_school_data = junior_high_school_data[junior_high_school_data['縣市名稱'] != '澎湖縣']


In [18]:
class_index = junior_high_school_data.columns[6:8+1]
class_index_english = ['7th-grade', '8th-grade', '9th-grade']
student_number_index = junior_high_school_data.columns[12:17+1]
for i in range(len(class_index)):
    plt.figure(figsize=(12, 4))
    # Remove empty schools
    junior_high_school_data_temp = junior_high_school_data[
        junior_high_school_data[class_index[i]] != 0]
    student_per_class = (junior_high_school_data_temp[student_number_index[2*i]] +
                         junior_high_school_data_temp[student_number_index[2*i+1]])/junior_high_school_data_temp[class_index[i]]
    n = plt.hist(student_per_class, bins=range(int(max(student_per_class))))
    probability = np.zeros(int(n[1].max())+1)
    probability[n[1][0:-1].astype(int)] = n[0]/sum(n[0])
    school_p[i+13] = probability

    plt.xticks(np.arange(0, int(max(student_per_class)), 5))
    plt.xlabel('Amount of ' + class_index_english[i] + ' student per class')
    plt.ylabel('Amount of school')
    plt.close()


### 1.2.3 Senior high school

In [19]:
senior_high_school_data = pd.read_excel(demographic_data_path/'高級中等學校校別資料檔.xls')
senior_high_school_data.columns = senior_high_school_data.iloc[1]
senior_high_school_data = senior_high_school_data[2::]
senior_high_school_data = senior_high_school_data[senior_high_school_data['縣市名稱'] != '金門縣']
senior_high_school_data = senior_high_school_data[senior_high_school_data['縣市名稱'] != '連江縣']
senior_high_school_data = senior_high_school_data[senior_high_school_data['縣市名稱'] != '澎湖縣']


In [20]:
# Remove 附設國中部
senior_high_school_data = senior_high_school_data[~(
    senior_high_school_data['學程(等級)別'] == 'J')]


In [21]:
class_index = senior_high_school_data.columns[9:11+1]
class_index_english = ['1st-grade', '2nd-grade', '3rd-grade']
student_number_index = senior_high_school_data.columns[16:21+1]
for i in range(len(class_index)):
    plt.figure(figsize=(12, 4))
    # Remove empty schools
    senior_high_school_data_temp = senior_high_school_data[
        senior_high_school_data[class_index[i]] != 0]
    student_per_class = (senior_high_school_data_temp[student_number_index[2*i]] +
                         senior_high_school_data_temp[student_number_index[2*i+1]])/senior_high_school_data_temp[class_index[i]]
    n = plt.hist(student_per_class, bins=range(int(max(student_per_class))))
    probability = np.zeros(int(n[1].max())+1)
    probability[n[1][0:-1].astype(int)] = n[0]/sum(n[0])
    school_p[i+16] = probability

    plt.xticks(np.arange(0, int(max(student_per_class)), 5))
    plt.xlabel('Amount of ' + class_index_english[i] + ' student per class')
    plt.ylabel('Amount of school')
    plt.close()


### 1.2.4 University

In [22]:
university_data = pd.read_excel(demographic_data_path/'大專校院各校科系別學生數.xlsx')
university_data.columns = university_data.iloc[1]
university_data = university_data[2::]
# Only extract bachelor students
university_data = university_data[university_data['等級別'] == 'B 學士']
university_data = university_data[university_data['縣市名稱'] != '71 金門縣']


In [23]:
# class_index = university_data.columns[9:22+1]
class_index_english = ['1st-grade', '2nd-grade', '3rd-grade', '4th-grade']
student_number_index = university_data.columns[9:16+1]
for i in range(len(class_index_english)):
    plt.figure(figsize=(12, 4))
    student_number = university_data[student_number_index[2*i]
                                     ] + university_data[student_number_index[2*i+1]]
    student_number = student_number[student_number > 0]  # Remove empty grade
    n = plt.hist(student_number, bins=range(int(max(student_number))))
    probability = np.zeros(int(n[1].max())+1)
    probability[n[1][0:-1].astype(int)] = n[0]/sum(n[0])
    school_p[i+19] = probability

    plt.xticks(np.arange(0, int(max(student_number)), 20))
    plt.xlabel('Amount of ' + class_index_english[i] + ' students')
    plt.ylabel('Amount of department')
    plt.close()


## 1.3 Workplace

In [24]:
workplace_p = {}


In [25]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18')
workplace_data = workplace_data.iloc[15:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)


In [26]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), number_of_companys)
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        # print(row.iloc[0])
        # plt.show()
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


In [27]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18-2')
workplace_data = workplace_data.iloc[14:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)


In [ ]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), row.iloc[3:14+1])
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        print(row.iloc[0])
        # plt.show()
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


In [29]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18-3')
workplace_data = workplace_data.iloc[14:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)

In [ ]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), row.iloc[3:14+1])
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        print(row.iloc[0])
        # plt.show()
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


In [31]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18-4')
workplace_data = workplace_data.iloc[14:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)

In [ ]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), row.iloc[3:14+1])
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        print(row.iloc[0])
        # plt.show()
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


In [33]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18-5')
workplace_data = workplace_data.iloc[14:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)

In [ ]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), row.iloc[3:14+1])
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        print(row.iloc[0])
        # plt.show()
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


In [35]:
workplace_data = pd.read_excel(demographic_data_path/'工業及服務業企業單位經營概況.xls', sheet_name='18-6')
workplace_data = workplace_data.iloc[14:-1, 1:17+1]
workplace_data = workplace_data.drop(['Unnamed: 11'], axis=1)

In [ ]:
headers = ['<5', '5~9', '10~19', '20~29', '30~39', '40~49',
           '50~99', '100~199', '200~299', '300~499', '500~999', '>1000']
for row in workplace_data.iloc:
    if type(row.iloc[0]) == str:
        plt.figure()
        number_of_companys = row.iloc[3:14+1]
        plt.bar(np.arange(len(headers)), row.iloc[3:14+1])
        plt.xticks(np.arange(len(headers)), headers, rotation=45)
        # plt.show()
        print(row.iloc[0])
        plt.close()
        probability = number_of_companys.to_numpy()/sum(number_of_companys.to_numpy())
        workplace_p_tmp = np.zeros(1000)
        # 1~999
        workplace_p_tmp[1:5] = probability[0]/4
        workplace_p_tmp[5:10] = probability[1]/5
        workplace_p_tmp[10:20] = probability[2]/10
        workplace_p_tmp[20:30] = probability[3]/10
        workplace_p_tmp[30:40] = probability[4]/10
        workplace_p_tmp[40:50] = probability[5]/10
        workplace_p_tmp[50:100] = probability[6]/50
        workplace_p_tmp[100:200] = probability[7]/100
        workplace_p_tmp[200:300] = probability[8]/100
        workplace_p_tmp[300:500] = probability[9]/200
        workplace_p_tmp[500:1000] = probability[10]/500
        # >1000
        if probability[11] != 0:
            append_numbers = np.ceil(probability[11]/workplace_p_tmp[-1])
            workplace_p_tmp = np.append(workplace_p_tmp, np.ones(
                int(append_numbers))*(probability[11]/append_numbers))
        workplace_p_tmp = np.array(workplace_p_tmp, dtype=float)

        workplace_p[row.iloc[0]] = workplace_p_tmp


## 1.4 Health care

In [37]:
health_care_data = pd.read_excel(demographic_data_path/'醫院平均每日醫療服務量統計.xls')
health_care_data.columns = health_care_data.iloc[3]
health_care_data = health_care_data[9::]
health_care_data = health_care_data.drop(['合計'], axis=1)


In [38]:
# Clean colunms and rows
health_care_data = health_care_data.drop(16)  # Drop 非公立醫院
health_care_data = health_care_data.drop(23)  # Drop 私立牙醫醫院
hospital_lists = health_care_data.iloc[:, 0].to_list()  # Save hospital names
hospital_numbers = health_care_data.iloc[:, 1].to_numpy()
# Clean data
for i in range(len(hospital_lists)):
    hospital_lists[i] = hospital_lists[i][2::]
health_care_data = health_care_data.iloc[:, 2:-1]

# Hospital name probability
hospital_p = health_care_data.sum(axis=1).to_numpy()
hospital_p = np.array(hospital_p/sum(hospital_p), dtype=float)
hospital_p = [hospital_lists, hospital_p]

hospital_size_p = {}
hospital_sizes = {}
for i, hospital_list in enumerate(hospital_lists):
    if sum(health_care_data.iloc[i].to_numpy()) > 0:
        hospital_size_p[hospital_list] = np.array(health_care_data.iloc[i].to_numpy(
        )/sum(health_care_data.iloc[i].to_numpy()), dtype=float)
    else:
        hospital_size_p[hospital_list] = 0
    hospital_sizes[hospital_list] = np.array(
        health_care_data.iloc[i].to_numpy()/hospital_numbers[i], dtype=float)


## 1.4 Municipality data

In [39]:
municipality_data = pd.read_excel(demographic_data_path/'municipality_data.xls', sheet_name='02-縣市別')
municipality_data.columns = municipality_data.iloc[1]
municipality_data = municipality_data[4:24+1]
municipality_data = municipality_data[municipality_data['區  域  別'] != '臺灣省']
municipality_data = municipality_data[municipality_data['區  域  別'] != '福建省']
municipality_data = municipality_data[municipality_data['區  域  別'] != '澎湖縣']
municipality_data = municipality_data[municipality_data['區  域  別'] != '金門縣']
municipality_data = municipality_data[municipality_data['區  域  別'] != '連江縣']
municipality_data = municipality_data[municipality_data['區  域  別'] != '東沙群島']
municipality_data = municipality_data[municipality_data['區  域  別'] != '南沙群島']

municipality_data = dict(
    zip(municipality_data['區  域  別'], municipality_data['人　　口　　數']))


In [ ]:
municipality_data

## 1.5 Age and gender

In [41]:
age_gender_data = pd.read_csv(demographic_data_path/'Single_Age_Population_2022-04-25.csv')
age_male_data = age_gender_data[age_gender_data['項目']
                                == '男性']['2022'].to_numpy()
age_female_data = age_gender_data[age_gender_data['項目']
                                  == '女性']['2022'].to_numpy()


In [ ]:
# Population pyramid
import plotly.graph_objects as gp

ages = np.arange(100+1)

# Creating instance of the figure
fig = gp.Figure()

# Adding Male data to the figure
fig.add_trace(gp.Bar(y=ages, x=age_male_data,
              name='Male', orientation='h'))

# Adding Female data to the figure
fig.add_trace(gp.Bar(y=ages, x=-age_female_data,
              name='Female', orientation='h'))

# Updating the layout for our graph
fig.update_layout(title='Population Pyramid of Taiwan 2022',
                  title_font_size=22, barmode='relative',
                  bargap=0.0, bargroupgap=0,
                  xaxis=dict(tickvals=[-100000, -200000, 0, 100000, 200000], ticktext=[
                             '100k', '200k', '0', '100k', '200k'], title='Population', title_font_size=14)
                  )

fig.show()


In [43]:
# Save age
age_p = np.array(age_male_data) + np.array(age_female_data)
age_p = age_p/np.sum(age_p)


In [44]:
# Save gender
gender_age_groups = np.vstack([age_male_data, age_female_data]).T
gender_p = np.empty([0, 2])
for i in gender_age_groups:
    gender_p = np.vstack([gender_p, i/sum(i)])


## 1.6 Student rate

In [45]:
# Decide the probability of the person is a student or not.
male_student_p = np.zeros(101)
female_student_p = np.zeros(101)


In [46]:
# Age 7 to 12 (elementary school)
male_elementary_student_p = elementary_school_data[[
    '1年級男', '2年級男', '3年級男', '4年級男', '5年級男', '6年級男']].sum().to_numpy()/age_male_data[7:12+1]
female_elementary_student_p = elementary_school_data[[
    '1年級女', '2年級女', '3年級女', '4年級女', '5年級女', '6年級女']].sum().to_numpy()/age_female_data[7:12+1]


male_student_p[7:12+1] = male_elementary_student_p
female_student_p[7:12+1] = female_elementary_student_p


In [47]:
# Age 13 to 15 (junior high school)
male_junior_high_school_student_p = junior_high_school_data[[
    '7年級男', '8年級男', '9年級男']].sum().to_numpy()/age_male_data[13:15+1]
female_junior_high_school_student_p = junior_high_school_data[[
    '7年級女', '8年級女', '9年級女']].sum().to_numpy()/age_female_data[13:15+1]

male_student_p[13:15+1] = male_junior_high_school_student_p
female_student_p[13:15+1] = female_junior_high_school_student_p


In [48]:
# Age 16 to 18 (senior high school)
male_senior_high_school_p = senior_high_school_data[[
    '一年級男', '二年級男', '三年級男']].sum().to_numpy()/age_male_data[16:18+1]
female_senior_high_school_p = senior_high_school_data[[
    '一年級女', '二年級女', '三年級女']].sum().to_numpy()/age_female_data[16:18+1]

male_student_p[16:18+1] = male_senior_high_school_p
female_student_p[16:18+1] = female_senior_high_school_p


In [49]:
# Age 19 to 22 (university)
male_university_p = university_data[[
    '一年級男生', '二年級男生', '三年級男生', '四年級男生']].sum().to_numpy()/age_male_data[19:22+1]
female_university_p = university_data[[
    '一年級女生', '二年級女生', '三年級女生', '四年級女生']].sum().to_numpy()/age_female_data[19:22+1]

male_student_p[19:22+1] = male_university_p
female_student_p[19:22+1] = female_university_p


In [50]:
# Replace >1 to 1
male_student_p[male_student_p > 1] = 1
female_student_p[female_student_p > 1] = 1

student_p = np.vstack([male_student_p, female_student_p])


## 1.7 Employment rate

In [51]:
# Emplolyment rate in each age group
employment_data = pd.read_excel(demographic_data_path/'就業率-年齡別-20220428.xlsx')
employment_data.columns = employment_data.iloc[2]
employment_data = employment_data.iloc[3::]


In [52]:
employment_p = employment_data.to_numpy()
employment_p = employment_p[0][1::]

male_employment_p = employment_p[0:9+1]/100
female_employment_p = employment_p[10::]/100

# Repeat emelent in the same age group
male_employment_p = np.repeat(male_employment_p, 5)  # Split age group
female_employment_p = np.repeat(female_employment_p, 5)  # Split age group

# Zero padding
male_employment_p = np.insert(
    male_employment_p, obj=0, values=np.zeros(15))  # Zero pading for age under 15
male_employment_p = np.append(male_employment_p, np.zeros(
    36))  # Append last value for age above 65
female_employment_p = np.insert(
    female_employment_p, obj=0, values=np.zeros(15))  # Zero pading for age under 15
female_employment_p = np.append(female_employment_p, np.zeros(
    36))  # Append last value for age above 65

employment_p = np.vstack([male_employment_p, female_employment_p])


## 1.8 Part-time and full-tim job probability

In [53]:
job_type_data = pd.read_excel(demographic_data_path/'mtable19.xlsx')
job_type_data = job_type_data.iloc[9:29]

# Drop 工業 and 服務業
job_type_data = job_type_data[~(
    job_type_data['Unnamed: 1'] == '工業\n  Goods-Producing Industries')]
job_type_data = job_type_data[~(
    job_type_data['Unnamed: 1'] == '服務業\n  Services-Producing Industries')]


In [54]:
male_job_type_data = job_type_data.iloc[:, 13:14+1]
female_job_type_data = job_type_data.iloc[:, 23:24+1]

job_list = job_type_data['Unnamed: 1'].to_numpy()

male_part_time_job_p = male_job_type_data['Unnamed: 14'].replace(
    '-', 0).to_numpy(dtype=float)/sum(male_job_type_data['Unnamed: 14'].replace('-', 0).to_numpy(dtype=float))
male_full_time_job_p = male_job_type_data['Unnamed: 13'].replace(
    '-', 0).to_numpy(dtype=float)/sum(male_job_type_data['Unnamed: 13'].replace('-', 0).to_numpy(dtype=float))

female_part_time_job_p = female_job_type_data['Unnamed: 24'].replace(
    '-', 0).to_numpy(dtype=float)/sum(female_job_type_data['Unnamed: 24'].replace('-', 0).to_numpy(dtype=float))
female_full_time_job_p = female_job_type_data['Unnamed: 23'].replace(
    '-', 0).to_numpy(dtype=float)/sum(female_job_type_data['Unnamed: 23'].replace('-', 0).to_numpy(dtype=float))

part_time_job_p = np.vstack([male_part_time_job_p, female_part_time_job_p])
full_time_job_p = np.vstack([male_full_time_job_p, female_full_time_job_p])


In [55]:
job_p = {}

# Clean job_list
for i in range(len(job_list)):
    job_list[i] = job_list[i].split('\n')[0].replace(' ', '')
job_list = job_list[job_list != '農、林、漁、牧業']
job_list = job_list[job_list != '公共行政及國防；強制性社會安全']
job_list[8] = '出版、影音製作、傳播及資通訊服務業'
job_list[9] = '金融及保險業、強制性社會安全'
job_list[13] = '教育業(註)'


job_p['job_list'] = job_list

part_time_job_p = np.delete(
    part_time_job_p, (0, 14), axis=1)  # Remove non exist job
row_sums = part_time_job_p.sum(axis=1)
part_time_job_p = part_time_job_p/row_sums[:, np.newaxis]
job_p['part_time_job_p'] = part_time_job_p

full_time_job_p = np.delete(
    full_time_job_p, (0, 14), axis=1)  # Remove non exist job
row_sums = full_time_job_p.sum(axis=1)
full_time_job_p = full_time_job_p/row_sums[:, np.newaxis]
job_p['full_time_job_p'] = full_time_job_p


## 1.9 Save demographic data

In [56]:
# population_size = 23215015
population_size = 23008366

In [57]:
demographic_parameters = [age_p, gender_p, student_p, employment_p, job_p, family_size_dict, municipality_data,
                school_p, workplace_p, hospital_p, hospital_size_p, hospital_sizes, population_size]


In [58]:
if save_p_state:
    with open('./variable/demographic_parameters.pkl', 'wb') as f:
        pickle.dump(demographic_parameters, f)


# 2. Course of disease parameters

## 2.1 Epidemic parameters

In [59]:
course_parameters = np.array([])
course_parameters_lb = np.array([])
course_parameters_ub = np.array([])

### 2.1.1 Latent period

Xin, Hualei, et al. "Estimating the latent period of coronavirus disease 2019 (COVID-19)." Clinical Infectious Diseases 74.9 (2022): 1678-1681.

In [60]:
latent_period_shape = 4.05
latent_period_shape_lb = 3.32
latent_period_shape_ub = 5.13

latent_period_scale = 1.35
latent_period_scale_lb = 1.06
latent_period_scale_ub = 1.67

course_parameters = np.append(course_parameters, (latent_period_shape, latent_period_scale))
course_parameters_lb = np.append(course_parameters_lb, (latent_period_shape_lb, latent_period_scale_lb))
course_parameters_ub = np.append(course_parameters_ub, (latent_period_shape_ub, latent_period_scale_ub))


### 2.1.2 Infectious period

Sanche, Steven, et al. "High contagiousness and rapid spread of severe acute respiratory syndrome coronavirus 2." Emerging infectious diseases 26.7 (2020): 1470.

In [61]:
infectious_period_shape = 4
infectious_period_shape_lb = 2
infectious_period_shape_ub = 6

infectious_period_mean = 10
infectious_period_mean_lb = 4
infectious_period_mean_ub = 14

infectious_period_scale = infectious_period_mean/infectious_period_shape
infectious_period_scale_lb = infectious_period_mean_lb/infectious_period_shape
infectious_period_scale_ub = infectious_period_mean_ub/infectious_period_shape


course_parameters = np.append(course_parameters, (infectious_period_shape, infectious_period_scale))
course_parameters_lb = np.append(course_parameters_lb, (infectious_period_shape_lb, infectious_period_scale_lb))
course_parameters_ub = np.append(course_parameters_ub, (infectious_period_shape_ub, infectious_period_scale_ub))



### 2.1.3 Incubation period

Cheng, Hao-Yuan, et al. "Contact tracing assessment of COVID-19 transmission dynamics in Taiwan and risk at different exposure periods before and after symptom onset." JAMA internal medicine 180.9 (2020): 1156-1163.

In [62]:
incubation_period_shape = 1.55
incubation_period_shape_lb = 0.73
incubation_period_shape_ub = 2.93

incubation_period_scale = 3.32
incubation_period_scale_lb = 1.6
incubation_period_scale_ub = 8.79

course_parameters = np.append(course_parameters, (incubation_period_shape, incubation_period_scale))
course_parameters_lb = np.append(course_parameters_lb, (incubation_period_shape_lb, incubation_period_scale_lb))
course_parameters_ub = np.append(course_parameters_ub, (incubation_period_shape_ub, incubation_period_scale_ub))

### 2.1.4 Fit Gamma distribution to days symptomatic to confirmed

In [63]:
# Load data
data_path = Path('./data/structured_course_of_disease_data')
Taiwan_data_sheet = pd.read_excel(data_path/'figshare_taiwan_covid.xlsx', 
                                  sheet_name=0)
Taiwan_data_sheet.columns = Taiwan_data_sheet.columns.str.strip().str.lower().str.\
    replace(' ', '_').str.replace('(', '').str.replace(')', '')

# Clean data
data_1_to_579 = clean_taiwan_data(Taiwan_data_sheet, 1, 579)

In [64]:
# Extract course of disease data
asymptomatic_to_symptom_days = extract_state_data(
    data_1_to_579, 'earliest_infection_date', 'onset_of_symptom')
asymptomatic_to_recover_days = extract_state_data(data_1_to_579, 'earliest_infection_date', 'recovery',
                                                  'onset_of_symptom')

symptom_to_icu_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'icu')
symptom_to_recover_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'recovery', 'icu')

icu_to_recover_days = extract_state_data(
    data_1_to_579, 'icu', 'recovery')
icu_to_dead_days = extract_state_data(
    data_1_to_579, 'icu', 'death_date')                                                 

asymptomatic_to_confirmed_days = extract_state_data(
    data_1_to_579, 'earliest_infection_date', 'confirmed_date', 'onset_of_symptom')
asymptomatic_to_confirmed_days = asymptomatic_to_confirmed_days.drop(asymptomatic_to_recover_days.index)
symptomatic_to_confirmed_days = extract_state_data(
    data_1_to_579, 'onset_of_symptom', 'confirmed_date')
symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_icu_days.index)
symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.drop(symptom_to_recover_days.index)

In [65]:
def gamma_fit_bootstrap(days_data, CI=0.68, allow_negative=False):
    n_bootstrap = 1000
    bootstrap_alphas = np.zeros(n_bootstrap)
    bootstrap_locs = np.zeros(n_bootstrap)
    bootstrap_scales = np.zeros(n_bootstrap)
    for i in tqdm(range(n_bootstrap)):
        bootstrap_sample = np.random.choice(days_data, size=len(days_data), replace=True)
        alpha, loc, scale = stats.gamma.fit(bootstrap_sample)
        if loc < 0:
            if allow_negative:
                pass
            else:
                
                alpha, loc, scale = stats.gamma.fit(bootstrap_sample, floc=0)
        bootstrap_alphas[i] = alpha
        bootstrap_locs[i] = loc
        bootstrap_scales[i] = scale

    alpha = np.median(bootstrap_alphas)
    alpha_lb = np.percentile(bootstrap_alphas, (1 - CI) / 2 * 100)
    alpha_ub = np.percentile(bootstrap_alphas, (1 + CI) / 2 * 100)
    loc = np.median(bootstrap_locs)
    loc_lb = np.percentile(bootstrap_locs, (1 - CI) / 2 * 100)
    loc_ub = np.percentile(bootstrap_locs, (1 + CI) / 2 * 100)
    scale = np.median(bootstrap_scales)
    scale_lb = np.percentile(bootstrap_scales, (1 - CI) / 2 * 100)
    scale_ub = np.percentile(bootstrap_scales, (1 + CI) / 2 * 100)

    return alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub

In [ ]:
if type(symptomatic_to_confirmed_days):
    symptomatic_to_confirmed_days = symptomatic_to_confirmed_days.apply(lambda x: x.days).to_numpy()
print('Number of cases:', len(symptomatic_to_confirmed_days))

CI = 0.68
alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(symptomatic_to_confirmed_days, CI)
symptom_to_confirmed_shape = alpha
symptom_to_confirmed_scale = scale
symptom_to_confirmed_loc = loc
print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(symptomatic_to_confirmed_days, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI*100}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Symptomatic to Confirmed Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))


### 2.1.5 Fit Gamma distribution to days asymptomatic to recovered

In [ ]:
# NOTE: Since we only have 2 data points, there is no point to do bootstraping. We just use the Gamma fit

if type(asymptomatic_to_recover_days) != np.ndarray:
    asymptomatic_to_recover_days = asymptomatic_to_recover_days.apply(lambda x: x.days).to_numpy()
print('Number of cases:', len(asymptomatic_to_recover_days))

alpha, loc, scale = stats.gamma.fit(asymptomatic_to_recover_days, floc=0)
# Set a 30% shift for alpha, loc, and scale
alpha_lb = alpha*0.8
alpha_ub = alpha*1.2
loc_lb = loc
loc_ub = loc
scale_lb = scale*0.8
scale_ub = scale*1.2
# alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(asymptomatic_to_recover_days, CI)
asymptomatic_to_recovered_shape = alpha
asymptomatic_to_recovered_scale = scale
asymptomatic_to_recovered_loc = loc

print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(asymptomatic_to_recover_days, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Asymptomatic to Recovered Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))


### 2.1.6 Fit Gamma distribution to days symptomatic to critically ill

In [ ]:
if type(symptom_to_icu_days) != np.ndarray:
    symptom_to_icu_days = symptom_to_icu_days.apply(lambda x: x.days).to_numpy()
# Fit Gamma distribution to days symptomatic to confirmed
# symptom_to_icu_days = symptom_to_icu_days[symptom_to_icu_days!=0] # To avoid floc=0 error, I ignore the 0 in the array.
print('Number of cases:', len(symptom_to_icu_days))

alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(symptom_to_icu_days, CI, allow_negative=True)
symptom_to_icu_shape = alpha
symptom_to_icu_scale = scale
symptom_to_icu_loc = loc

print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(symptom_to_icu_days, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Symptomatic to Critically illed Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))


### 2.1.7 Fit Gamma distribution to days symptomatic to recovered

In [ ]:
# Only 24 data points. We just use the Gamma fit

if  type(symptom_to_recover_days) != np.ndarray:
    symptom_to_recover_days = symptom_to_recover_days.apply(lambda x: x.days).to_numpy()

print('Number of cases:', len(symptom_to_recover_days))
print()

alpha, loc, scale = stats.gamma.fit(symptom_to_recover_days)
# alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(symptom_to_recover_days, CI)
alpha_lb = alpha*0.8
alpha_ub = alpha*1.2
loc_lb = loc*0.8
loc_ub = loc*1.1
scale_lb = scale*0.8
scale_ub = scale*1.2

symptom_to_recover_shape = alpha
symptom_to_recover_scale = scale
symptom_to_recover_loc = loc

print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(symptom_to_recover_days, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI*100}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Symptomatic to Recovered Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))


### 2.1.8 Fit Gamma distribution to days critically ill to recovered

In [ ]:
# NOTE: It's only 24 data points. We just use the Gamma fit

if  type(icu_to_recover_days) != np.ndarray:
    icu_to_recover_days = icu_to_recover_days.apply(lambda x: x.days).to_numpy()

print('Number of cases:', len(icu_to_recover_days))
print()

alpha, loc, scale = stats.gamma.fit(icu_to_recover_days)
# alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(icu_to_recover_days, CI)
alpha_lb = alpha*0.8
alpha_ub = alpha*1.2
loc_lb = loc*0.8
loc_ub = loc*1.2
scale_lb = scale*0.8
scale_ub = scale*1.2

icu_to_recover_shape = alpha
icu_to_recover_scale = scale
icu_to_recover_loc = loc

print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(icu_to_recover_days, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI*100}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Critically to Recovered Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))


### 2.1.9 Infection to death

Parameters from "Ward, T., & Johnsen, A. (2021). Understanding an evolving pandemic: An analysis of the clinical time delay distributions of COVID-19 in the United Kingdom. Plos one, 16(10), e0257978."

In [71]:
infection_to_death_shape = 3.07
infection_to_death_shape_lb = 2.96
infection_to_death_shape_ub = 3.17

infection_to_death_scale = 1/0.15
infection_to_death_scale_lb = 6.25
infection_to_death_scale_ub = 6.67

course_parameters = np.append(course_parameters, (infection_to_death_shape, infection_to_death_scale))
course_parameters_lb = np.append(course_parameters_lb, (infection_to_death_shape_lb, infection_to_death_scale_lb))
course_parameters_ub = np.append(course_parameters_ub, (infection_to_death_shape_ub, infection_to_death_scale_ub))

### 2.1.10 Fit Gamma distribution to days from negative test to positive test

In [72]:
negative_test_date_1 = pd.to_timedelta(extract_state_data(
    data_1_to_579, 'confirmed_date', 'negative_test_date_1'))
negative_test_date_2 = pd.to_timedelta(extract_state_data(
    data_1_to_579, 'confirmed_date', 'negative_test_date_2'))
negative_test_date_3 = pd.to_timedelta(extract_state_data(
    data_1_to_579, 'confirmed_date', 'negative_test_date_3'))
negative_test_date_4 = pd.to_timedelta(extract_state_data(
    data_1_to_579, 'confirmed_date', 'negative_test_date_4'))

negative_test_date_array = np.hstack([negative_test_date_1.dt.days.to_numpy(), negative_test_date_2.dt.days.to_numpy(
), negative_test_date_3.dt.days.to_numpy(), negative_test_date_4.dt.days.to_numpy()])

In [ ]:
print('Number of cases:', len(negative_test_date_array))

alpha, alpha_lb, alpha_ub, loc, loc_lb, loc_ub, scale, scale_lb, scale_ub = gamma_fit_bootstrap(-negative_test_date_array, CI)
negative_to_confirmed_shape = alpha
negative_to_confirmed_scale = scale
negative_to_confirmed_loc = loc

print(f"alpha: {alpha:.4f} ({alpha_lb:.4f}, {alpha_ub:.4f})")
print(f"loc: {loc:.4f} ({loc_lb:.4f}, {loc_ub:.4f})")
print(f"scale: {scale:.4f} ({scale_lb:.4f}, {scale_ub:.4f})")

# Plot the fitted distribution with confidence interval
x = np.linspace(0, 80, 1000)
y = stats.gamma.pdf(x, alpha, loc, scale)

# Calculate confidence intervals
y_lower = stats.gamma.pdf(x, alpha_lb, loc_lb, scale_lb)
y_upper = stats.gamma.pdf(x, alpha_ub, loc_ub, scale_ub)

plt.figure()
plt.hist(-negative_test_date_array, density=True, alpha=0.7, label='Taiwanese data')
plt.plot(x, y, 'r-', lw=2, label='Fitted Gamma distribution')
plt.fill_between(x, y_lower, y_upper, color='r', alpha=0.2, label=f'{CI*100}% CI')
plt.xlabel('Days')
plt.ylabel('Density')
plt.title('Negative test to Confirmed Days')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


course_parameters = np.append(course_parameters, (alpha, scale, loc))
course_parameters_lb = np.append(course_parameters_lb, (alpha_lb, scale_lb, loc_lb))
course_parameters_ub = np.append(course_parameters_ub, (alpha_ub, scale_ub, loc_ub))

### 2.1.11 Age dependent risk ratio from Cheng2020

In [74]:
age_risk_ratios = np.array([0.3, 1, 2.19, 1.75]) # For age 0-19, 20-39, 40-59, and 60 above. Note that for age 0-19, I set it to be 0.3 by 1/281
age_risk_ratios_lb = np.array([0, 0, 0.78, 0.44])
age_risk_ratios_ub = np.array([1, 2, 6.14, 6.97])


course_parameters = np.append(course_parameters, age_risk_ratios)
course_parameters_lb = np.append(course_parameters_lb, age_risk_ratios_lb)
course_parameters_ub = np.append(course_parameters_ub, age_risk_ratios_ub)

### 2.1.12 Natural immunity

Pilz, S., Chakeri, A., Ioannidis, J. P., Richter, L., Theiler‐Schwetz, V., Trummer, C., ... & Allerberger, F. (2021). SARS‐CoV‐2 re‐infection risk in Austria. European journal of clinical investigation, 51(4), e13520.

In [75]:
natural_immunity_rate = 0.91  
natural_immunity_rate_lb = 0.87
natural_immunity_rate_ub = 0.93

course_parameters = np.append(course_parameters, natural_immunity_rate)
course_parameters_lb = np.append(course_parameters_lb, natural_immunity_rate_lb)
course_parameters_ub = np.append(course_parameters_ub, natural_immunity_rate_ub)

### 2.1.12 Vaccine efficacy

Article_Dhayfule2021_Vaccine_review_210514

Figure 6: Vaccine efficacy against SARS-CoV-2 asymptomatic with 95% confidence interval.

In [76]:
vaccine_rate = 0 # randomly set
vaccine_rate_lb = 0
vaccine_rate_ub = 0

vaccine_efficacy = 0.905
vaccine_efficacy_lb = 0.881
vaccine_efficacy_ub = 0.924

course_parameters = np.append(course_parameters, (vaccine_rate, vaccine_efficacy))
course_parameters_lb = np.append(course_parameters_lb, (vaccine_rate_lb, vaccine_efficacy_lb))
course_parameters_ub = np.append(course_parameters_ub, (vaccine_rate_ub, vaccine_efficacy_ub))

### 2.1.13 Attack rate, data extracted from Cheng2020 and Ge2021, time range -14 to 10

- Workplace attack rate is from Chen2021_COVID-19 
- Class attack rate is from Huang2021_Estimation

In [77]:
attack_rate = np.array([0.86, 0.98, 1.09, 1.16, 1.16, 1.07, 0.95, 0.84, 0.78, 0.78, 0.86,
                       1.02, 1.18, 1.3, 1.34, 1.33, 1.27, 1.19, 1.11, 1.05, 1.01, 0.99, 0.98, 0.99, 0.99]) # Relative risk
shift = np.mean(attack_rate) -1 # Mean of the relative risk should be 1
attack_rate = attack_rate - shift
attack_rate_lb = np.array([0.39, 0.63, 0.9, 0.91, 0.81, 0.75, 0.72, 0.7, 0.65, 0.63, 0.72, 
                           0.9, 1.05, 1.13, 1.18, 1.19, 1.15, 1.07, 0.98, 0.92, 0.89, 0.88, 0.84, 0.76, 0.69]) - shift
attack_rate_ub = np.array([1.91, 1.52, 1.31, 1.48, 1.66, 1.54, 1.26, 1.02, 0.94, 0.96, 1.04,
                           1.15, 1.34, 1.49, 1.54, 1.49, 1.4, 1.32, 1.26, 1.2, 1.14, 1.11, 1.15, 1.27, 1.43]) - shift
# attack_rate = np.array([0.78, 0.78, 0.86,
#                        1.02, 1.18, 1.3, 1.34, 1.33, 1.27, 1.19, 1.11, 1.05, 1.01, 0.99, 0.98, 0.99, 0.99])

household_attack_rate = attack_rate*0.101
household_attack_rate_lb = attack_rate_lb*0.101
household_attack_rate_ub = attack_rate_ub*0.101

school_attack_rate = attack_rate*0.024
school_attack_rate_lb = attack_rate_lb*0.024
school_attack_rate_lb = school_attack_rate_lb * (0.01 / np.mean(school_attack_rate_lb))
school_attack_rate_ub = attack_rate_ub*0.024
school_attack_rate_ub = school_attack_rate_ub * (0.04 / np.mean(school_attack_rate_ub))

workplace_attack_rate = attack_rate*0.034
workplace_attack_rate_lb = attack_rate_lb*0.034
workplace_attack_rate_lb = workplace_attack_rate_lb * (0.01 / np.mean(workplace_attack_rate_lb))
workplace_attack_rate_ub = attack_rate_ub*0.034
workplace_attack_rate_ub = workplace_attack_rate_ub * (0.15 / np.mean(workplace_attack_rate_ub))

health_care_attack_rate = attack_rate*0.004
health_care_attack_rate_lb = attack_rate_lb*0.004
health_care_attack_rate_lb = health_care_attack_rate_lb * (0.001 / np.mean(health_care_attack_rate_lb))
health_care_attack_rate_ub = attack_rate_ub*0.004
health_care_attack_rate_ub = health_care_attack_rate_ub * (0.016 / np.mean(health_care_attack_rate_ub))

municipality_attack_rate = (attack_rate/np.mean(attack_rate))*0.002
municipality_attack_rate_lb = (attack_rate_lb/np.mean(attack_rate))*0.002
municipality_attack_rate_lb = municipality_attack_rate_lb * (0.001 / np.mean(municipality_attack_rate_lb))
municipality_attack_rate_ub = (attack_rate_ub/np.mean(attack_rate))*0.002
municipality_attack_rate_ub = municipality_attack_rate_ub * (0.01 / np.mean(municipality_attack_rate_ub))

course_parameters = np.append(course_parameters, (household_attack_rate, school_attack_rate, workplace_attack_rate, 
                                                  health_care_attack_rate, municipality_attack_rate))
course_parameters_lb = np.append(course_parameters_lb, (household_attack_rate_lb, school_attack_rate_lb, workplace_attack_rate_lb, 
                                                        health_care_attack_rate_lb, municipality_attack_rate_lb))
course_parameters_ub = np.append(course_parameters_ub, (household_attack_rate_ub, school_attack_rate_ub, workplace_attack_rate_ub, 
                                                        health_care_attack_rate_ub, municipality_attack_rate_ub))


## 2.2 Transition probability

In [78]:
shift_percentage = 0.3

In [79]:
infection_to_recovered_transition_p = 0.09 # Cheng et al.
symptom_to_recovered_transition_p = 1-0.18 # https://www.cna.com.tw/news/ahel/202210285003.aspx, https://www.thenewslens.com/article/154421, Taiwan_critically_ill_rate.png
critically_ill_to_recovered_transition_p = len(icu_to_recover_days)/(
    len(icu_to_recover_days)+len(icu_to_dead_days))

transition_p = np.array([infection_to_recovered_transition_p, symptom_to_recovered_transition_p,
                        critically_ill_to_recovered_transition_p])
transition_p_lb = transition_p-transition_p*shift_percentage
transition_p_lb[transition_p_lb<0] = 0
transition_p_ub = transition_p+transition_p*shift_percentage
transition_p_ub[transition_p_ub>1] = 1

course_parameters = np.hstack([course_parameters, transition_p])
course_parameters_lb = np.hstack([course_parameters_lb, transition_p_lb])
course_parameters_ub = np.hstack([course_parameters_ub, transition_p_ub])

In [ ]:
print(f'Infection to recovered: {infection_to_recovered_transition_p: .4f}')
print(f'Symptom to recovered: {symptom_to_recovered_transition_p: .4f}')
print(f'Critically ill to recovered: {critically_ill_to_recovered_transition_p: .4f}')

In [ ]:
print(transition_p_lb)
print(transition_p_ub)

In [ ]:
# Check if course_parameters stay within course_parameters_lb and course_parameters_ub
within_bounds = np.logical_and(course_parameters >= course_parameters_lb, 
                               course_parameters <= course_parameters_ub)

if np.all(within_bounds):
    print("All course_parameters are within their respective bounds.")
else:
    out_of_bounds = np.where(~within_bounds)[0]
    print(f"Warning: {len(out_of_bounds)} parameter(s) are out of bounds:")
    for idx in out_of_bounds:
        print(f"Parameter {idx}: {course_parameters[idx]:.4f} "
              f"(bounds: {course_parameters_lb[idx]:.4f}, {course_parameters_ub[idx]:.4f})")

# Visualize the parameters and their bounds
plt.figure(figsize=(12, 6))
x = np.arange(len(course_parameters))
plt.errorbar(x, course_parameters, 
             yerr=[course_parameters - course_parameters_lb, 
                   course_parameters_ub - course_parameters],
             fmt='.', capsize=5, capthick=2, ecolor='red', label='Bounds')
plt.plot(x, course_parameters, 'b-', label='Parameters')
plt.xlabel('Parameter Index')
plt.ylabel('Value')
plt.title('Course Parameters with Bounds')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()


In [83]:
if save_p_state:
    with open('./variable/course_parameters.npy', 'wb') as f:
        np.save(f, course_parameters)
    with open('./variable/course_parameters_lb.npy', 'wb') as f:
        np.save(f, course_parameters_lb)
    with open('./variable/course_parameters_ub.npy', 'wb') as f:
        np.save(f, course_parameters_ub)

In [ ]:
len(course_parameters)

# 3. Social contact probability boundary

In [85]:
contact_parameters = {
    'household_lower_bound': [0.95, 0.4, 0.05, 0.01, 1, 0, 0],
    'household_upper_bound': [1, 1, 0.5, 0.1, 20, 10, 10],

    'school_lower_bound': [0.1, 0.5, 0.01, 0.001, 1, 0, 0],
    'school_upper_bound': [1, 1, 0.5, 0.05, 10, 10, 10],

    'workplace_lower_bound': [0.1, 0.5, 0.01, 0.001, 1, 0, 0],
    'workplace_upper_bound': [1, 1, 0.5, 0.05, 10, 10, 10],

    'health_care_lower_bound': [0.005, 0.5, 0.001, 0.6, 1, 0, 0],
    'health_care_upper_bound': [0.008, 1, 0.006, 1, 10, 10, 10],

    'municipality_lower_bound': [0.000002, 0.5, 0.01, 0.001, 1, 0, 0],
    'municipality_upper_bound': [0.00001, 1, 0.1, 0.05, 10, 10, 10],

    'overdispersion_lower_bound': [0, 1],
    'overdispersion_upper_bound': [0.2, 20]
}

if save_p_state:
    with open('./variable/contact_parameters.pkl', 'wb') as f:
        pickle.dump(contact_parameters, f)


# 4. Firefly results

## 4.1 Load optimization results

In [86]:
folder = Path('./firefly_result/Firefly_result_pop_size_100_alpha_1_betamin_1_gamma_0.131_max_generations_200')
file_name = 'firefly_best.txt'

# Load best fireflies
with open(folder/file_name, 'r') as f:
    best_results = [[float(num) for num in line.split(' ')] for line in f]

best_results = np.array(best_results)
# best_iterations = best_results[:, 0]
fireflies = best_results[:, 1:-1]
costs = best_results[:, -1]

# Get the 10% best fireflies
top_10_percent = int(len(costs)*0.1)
sorted_indices = np.argsort(costs)
best_cost_index = sorted_indices[:top_10_percent]

best_costs = costs[best_cost_index]
best_fireflies = fireflies[best_cost_index, :]


# Load bound
file_name = 'bound.txt'
# file_name = 'C:/Users/chee8/Desktop/to_be_deleted/bound.txt'
with open(folder/file_name, 'r') as f:
    bounds = [[float(num) for num in line.split(' ')] for line in f]
bounds = np.array(bounds)
lower_bound = bounds[0, :]
upper_bound = bounds[1, :]


## 4.2 Firefly results of the social contact probability

In [ ]:
# Claude code
# Social contact probability
social_contact_firefly_result_dict = {}
social_contact_firefly_lower_bound_dict = {}
social_contact_firefly_upper_bound_dict = {}
contact_layers = ['Household', 'School', 'Workplace', 'Heath care', 'Municipality']
parameters = [
    'contact probability',
    'consecutive daily contact probability',
    'contact probability when health',
    'contact probability when symptomatic',
    'steepness',
    'phase relative to symptom-onset for symptomatic',
    'phase relative to symptomatic for resuming normal social context'
]
extra_parameters = ['overdispersion rate', 'overdispersion weight']

pe_social_contact_p = pd.DataFrame()

# Function to check if a value is within bounds
def is_within_bounds(value, lower, upper):
    return lower <= value <= upper

# Flag to track if any firefly is out of bounds
any_firefly_out_of_bounds = False

for i in range(top_10_percent):
    firefly_within_bounds = True
    for contact_layer in contact_layers:
        for param in parameters:
            key = f'{contact_layer} {param}'
            idx = contact_layers.index(contact_layer) * len(parameters) + parameters.index(param)
            
            value = best_fireflies[i, idx]
            lower = lower_bound[idx]
            upper = upper_bound[idx]
            
            if not is_within_bounds(value, lower, upper):
                print(f"Warning: Firefly {i+1}, {key} is out of bounds. Value: {value}, Bounds: [{lower}, {upper}]")
                firefly_within_bounds = False
                any_firefly_out_of_bounds = True
            
            social_contact_firefly_result_dict[key] = value
            social_contact_firefly_lower_bound_dict[key] = lower
            social_contact_firefly_upper_bound_dict[key] = upper
    for extra_parm in extra_parameters:
        key = f'{extra_parm}'
        idx += 1

        value = best_fireflies[i, idx]
        lower = lower_bound[idx]
        upper = upper_bound[idx]
        if not is_within_bounds(value, lower, upper):
                print(f"Warning: Firefly {i+1}, {key} is out of bounds. Value: {value}, Bounds: [{lower}, {upper}]")
                firefly_within_bounds = False
                any_firefly_out_of_bounds = True
            
        social_contact_firefly_result_dict[key] = value
        social_contact_firefly_lower_bound_dict[key] = lower
        social_contact_firefly_upper_bound_dict[key] = upper
    
    if not firefly_within_bounds:
        print(f"Firefly {i+1} is out of bounds for one or more parameters.")
    
    pe_social_contact_p[f'Optimized value {i+1}'] = pd.Series(social_contact_firefly_result_dict)

pe_social_contact_p['Lower bound'] = pd.Series(social_contact_firefly_lower_bound_dict)
pe_social_contact_p['Upper bound'] = pd.Series(social_contact_firefly_upper_bound_dict)

pe_social_contact_p.to_excel('./variable/pe_social_contact_parameters.xlsx')

if any_firefly_out_of_bounds:
    print("Warning: One or more fireflies were found to be out of bounds.")
else:
    print("All fireflies are within the specified boundaries.")

## 4.3 Firefly results of the course of disease prameters

In [88]:
course_var_name = [
    'Latent period, Gamma shape', 'Latent period, Gamma scale',
    'Infectious period, Gamma shape', 'Infectious period, Gamma scale',
    'Incubation period, Gamma shape', 'Incubation period, Gamma scale',
    'Symptom onset to confirmed period, Gamma shape', 'Symptom onset to confirmed period, Gamma scale',
    'Symptom onset to confirmed period, Gamma loc', 'Asymptomatic to recovered period, Gamma shape',
    'Asymptomatic to recovered period, Gamma scale', 'Asymptomatic to recovered period, Gamma loc',
    'Symptom onset to critically ill period, Gamma shape', 'Symptom onset to critically ill period, Gamma scale', 'Symptom onset to critically ill period, Gamma loc',
    'Symptom onset to recovered period, Gamma shape', 'Symptom onset to recovered period, Gamma scale', 'Symptom onset to recovered period, Gamma loc',
    'Critically ill to recovered period, Gamma shape', 'Critically ill to recovered period, Gamma scale',
    'Critically ill to recovered period, Gamma loc', 'Asymptomatic to death period, Gamma shape',
    'Asymptomatic to death period, Gamma scale', 'Negative test to confirmed period, Gamma shape',
    'Negative test to confirmed period, Gamma scale', 'Negative test to confirmed period, Gamma loc'
]
# tmp = [f'Age risk ratio {i}' for i in range(4)]
tmp = ['Age risk ratio, 0-19', 'Age risk ratio, 20-39', 'Age risk ratio, 40-59', 'Age risk ratio, >=60']
course_var_name = course_var_name + tmp
tmp = ['Natural immunity rate', 'Vaccination rate', 'Vaccine efficacy']
course_var_name = course_var_name + tmp
tmp = [f'Household secondary attack rate relative risk: {i} days from onset of symptom to contact date' for i in range(-14, 10+1)]
course_var_name = course_var_name + tmp
tmp = [f'School secondary attack rate relative risk: {i} days from onset of symptom to contact date' for i in range(-14, 10+1)]
course_var_name = course_var_name + tmp
tmp = [f'Workplace secondary attack rate relative risk: {i} days from onset of symptom to contact date' for i in range(-14, 10+1)]
course_var_name = course_var_name + tmp
tmp = [f'Health care secondary attack rate relative risk: {i} days from onset of symptom to contact date' for i in range(-14, 10+1)]
course_var_name = course_var_name + tmp
tmp = [f'Municipality secondary attack rate relative risk: {i} days from onset of symptom to contact date' for i in range(-14, 10+1)]
course_var_name = course_var_name + tmp
tmp = ['Infection to recovered transition p', 'Symptomatic to recovered transition p',
       'Critically ill to recovered transition p']
course_var_name = course_var_name + tmp

In [89]:
pe_course_of_disease_p = pd.DataFrame()
for i in range(len(best_fireflies)):
    # Assign value
    course_firefly_result_dict = dict(zip(course_var_name, best_fireflies[i, 37:]))
    pe_course_of_disease_p[f'Optimized value {i+1}'] = pd.DataFrame.from_dict(course_firefly_result_dict, orient='index', columns=[f'Optimized value {i+1}'])

In [90]:
course_firefly_lower_bound_dict = dict(zip(course_var_name, lower_bound[37::]))
course_firefly_upper_bound_dict = dict(zip(course_var_name, upper_bound[37::]))
pe_course_of_disease_p['Lower bound'] = pd.DataFrame.from_dict(course_firefly_lower_bound_dict, orient='index', columns=['Lower bound'])
pe_course_of_disease_p['Upper bound'] = pd.DataFrame.from_dict(course_firefly_upper_bound_dict, orient='index', columns=['Upper bound'])

In [ ]:
# Insert the initial value and the corresponding reference
pe_course_of_disease_p['Initial value'] = '-'
pe_course_of_disease_p['Reference 1'] = '-'
pe_course_of_disease_p['Reference 2'] = '-'
init_course_of_disease_p = pd.read_excel(Path('./variable/init_course_of_disease_parameters.xlsx'), sheet_name=0)
layer_attack_rate = [10.1/(100*np.mean(attack_rate)), 2.4/(100*np.mean(attack_rate)), 
                    3.4/(100*np.median(attack_rate)), 0.4/(100*np.mean(attack_rate)), 
                    0.2/(100*np.mean(attack_rate))]

layers = ['Household', 'School', 'Workplace', 'Health care', 'Municipality']
for i in range(len(init_course_of_disease_p)):
    variable = init_course_of_disease_p.loc[i, 'Variable']
    if variable in pe_course_of_disease_p.index:
        pe_course_of_disease_p.loc[variable, 'Initial value'] = init_course_of_disease_p.loc[i, 'Value']
        pe_course_of_disease_p.loc[variable, 'Reference 1'] = init_course_of_disease_p.loc[i, 'Reference']
    print()
    if variable[:9] == 'Secondary':
        for j, layer in enumerate(layers):
            tmp = layer + ' ' + variable.lower()
            pe_course_of_disease_p.loc[tmp, 'Initial value'] = init_course_of_disease_p.loc[i, 'Value']*layer_attack_rate[j]
            pe_course_of_disease_p.loc[tmp, 'Reference 1'] = init_course_of_disease_p.loc[i, 'Reference']
            if layer == 'Household':
                pe_course_of_disease_p.loc[tmp, 'Reference 2'] = 'Ge, Y., Martinez, L., Sun, S., Chen, Z., Zhang, F., Li, F., Sun, W., Chen, E., Pan, J., Li, C., et al.: Covid-19 transmission dynamics among close contacts of index patients with covid-19: a population-based cohort study in zhejiang province, china. JAMA Internal Medicine 181(10), 1343–1350 (2021)'
            elif layer == 'School':
                pe_course_of_disease_p.loc[tmp, 'Reference 2'] = 'Huang, Y.-T., Tu, Y.-K., Lai, P.-C.: Estimation of the secondary attack rate of covid-19 using proportional meta-analysis of nationwide contact tracing data in taiwan. Journal of Microbiology, Immunology and Infection 54(1), 89–92 (2021)'
            elif layer == 'Workplace':
                pe_course_of_disease_p.loc[tmp, 'Reference 2'] = 'Chen, Yiqun, et al. "COVID-19 outbreak rates and infection attack rates associated with the workplace: a descriptive epidemiological study." BMJ open 12.7 (2022): e055643.'
            elif layer == 'Health care':
                pe_course_of_disease_p.loc[tmp, 'Reference 2'] = 'Ge, Y., Martinez, L., Sun, S., Chen, Z., Zhang, F., Li, F., Sun, W., Chen, E., Pan, J., Li, C., et al.: Covid-19 transmission dynamics among close contacts of index patients with covid-19: a population-based cohort study in zhejiang province, china. JAMA Internal Medicine 181(10), 1343–1350 (2021)'

In [92]:
# Add the fitted Gamma distribution parameters to the table
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma shape', 'Initial value'] = symptom_to_confirmed_shape
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma scale', 'Initial value'] = symptom_to_confirmed_scale
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma loc', 'Initial value'] = symptom_to_confirmed_loc
pe_course_of_disease_p.loc['Symptom onset to confirmed period, Gamma loc', 'Reference 1'] = 'Determined through TW data'

pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma shape', 'Initial value'] = asymptomatic_to_recovered_shape
pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma scale', 'Initial value'] = asymptomatic_to_recovered_scale
pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma loc', 'Initial value'] = asymptomatic_to_recovered_loc
pe_course_of_disease_p.loc['Asymptomatic to recovered period, Gamma loc', 'Reference 1'] = 'Determined through TW data'

pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma shape', 'Initial value'] = symptom_to_icu_shape
pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma scale', 'Initial value'] = symptom_to_icu_scale
pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma loc', 'Initial value'] = symptom_to_icu_loc
pe_course_of_disease_p.loc['Symptom onset to critically ill period, Gamma loc', 'Reference 1'] = 'Determined through TW data'


pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma shape', 'Initial value'] = symptom_to_recover_shape
pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma scale', 'Initial value'] = symptom_to_recover_scale
pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma loc', 'Initial value'] = symptom_to_recover_loc
pe_course_of_disease_p.loc['Symptom onset to recovered period, Gamma loc', 'Reference 1'] = 'Determined through TW data'

pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma shape', 'Initial value'] = icu_to_recover_shape
pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma scale', 'Initial value'] = icu_to_recover_scale
pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma loc', 'Initial value'] = icu_to_recover_loc
pe_course_of_disease_p.loc['Critically ill to recovered period, Gamma loc', 'Reference 1'] = 'Determined through TW data'

pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma shape', 'Initial value'] = negative_to_confirmed_shape
pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma shape', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma scale', 'Initial value'] = negative_to_confirmed_scale
pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma scale', 'Reference 1'] = 'Determined through TW data'
pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma loc', 'Initial value'] = negative_to_confirmed_loc
pe_course_of_disease_p.loc['Negative test to confirmed period, Gamma loc', 'Reference 1'] = 'Determined through TW data'


pe_course_of_disease_p.loc['Infection to recovered transition p', 'Initial value'] = infection_to_recovered_transition_p
pe_course_of_disease_p.loc['Infection to recovered transition p', 'Reference 1'] = 'Cheng, H.-Y., Jian, S.-W., Liu, D.-P., Ng, T.-C., Huang, W.-T., Lin, H.-H.: Contact Tracing Assessment of COVID-19 Transmission Dynamics in Taiwan and Risk at Diff erent Exposure Periods Before and After Symptom Onset. JAMA Internal Medicine 180(9), 1156 (2020).'
pe_course_of_disease_p.loc['Symptomatic to recovered transition p', 'Initial value'] = symptom_to_recovered_transition_p
pe_course_of_disease_p.loc['Symptomatic to recovered transition p', 'Reference 1'] = 'https://www.cna.com.tw/news/ahel/202210285003.aspx'
pe_course_of_disease_p.loc['Symptomatic to recovered transition p', 'Reference 2'] = 'https://www.thenewslens.com/article/154421'
pe_course_of_disease_p.loc['Critically ill to recovered transition p', 'Initial value'] = critically_ill_to_recovered_transition_p
pe_course_of_disease_p.loc['Critically ill to recovered transition p', 'Reference 1'] = 'Determined through TW data'


In [93]:
pe_course_of_disease_p.to_excel('./variable/pe_course_of_disease_parameters.xlsx')

# 5. Taiwan data test matrix

## 5.1 Convert Taiwan individual data to test matrix

In [94]:
data_matrix = convert_Taiwan_data_to_test_matrix(data_1_to_579)

In [ ]:
# Check non-NaN counts
value_counts = np.sum(~np.isnan(data_matrix), axis=1)
plt.hist(value_counts, bins=range(max(value_counts)+2))
plt.xlabel('Number of non-NaN elements each subject in the Taiwan data matrix')
plt.ylabel('Frequency')
# print(f'{np.unique(value_counts, return_counts=True)}')
print('number of non-nan element in each column: ', np.sum(~np.isnan(data_matrix), axis=0))


# plt.savefig('Taiwan_matrix_non_nan_RW2022.pdf')

## 5.2 Contact network

Check plot_taiwan_data.ipynb to see detail code for constructing the contact network

In [96]:
import networkx as nx

In [97]:
edge_list = pd.read_excel(data_path/'figshare_taiwan_covid.xlsx', 
                                  sheet_name='Edge List')

### 5.2.1 Uninfected contacts for each subject

In [98]:
case_ids = np.arange(579, 0, -1)
# case_ids = np.arange(579, 570, -1)
remove_id = 530
number_of_contacts = np.array([])
number_of_infected_contacts = np.array([])
for case_id in case_ids:
    if case_id == remove_id:
        continue
    else:
        try:
            contact_nodes = list(taiwan_contact_network['I'+str(case_id)])
        except:
            contact_nodes = []
        number_of_contact = len(contact_nodes)
        number_of_contacts = np.append(number_of_contacts, number_of_contact)
        number_of_infected_contact = np.sum([True if type(i) == str and i[0]=='I' else False for i in contact_nodes])
        number_of_infected_contacts = np.append(number_of_infected_contacts, number_of_infected_contact)

number_of_uninfected_contacts = number_of_contacts - number_of_infected_contacts
number_of_uninfected_contacts[number_of_uninfected_contacts==0] = np.nan # When it is 0, it 
# might because the CDC didn't release the data so that we should set it to nan.

### 5.2.2 Effective contacts for each subject

Note that number_of_effective_contacts is different from number_of_infected_contact. Number_of_effective_contacts is focusing on the contact network with clear infection path while number_of_infected_contact is only showing the connection within infected cases. Some time the connection within the infected cases is unclear because CDC only show the infection cluster so we don't know how many people are infected.

In [99]:
case_ids = np.arange(579, 0, -1)
# case_ids = np.arange(579, 570, -1)
remove_id = 530
number_of_effective_contacts = np.array([])
for case_id in case_ids:
    if case_id == remove_id:
        continue
    else:
        try:
            contact_nodes = list(infection_contact_network['I'+str(case_id)])
        except:
            contact_nodes = []
        number_of_effective_contact = len(contact_nodes)
        number_of_effective_contacts = np.append(number_of_effective_contacts, number_of_effective_contact)

number_of_effective_contacts[number_of_effective_contacts==0] = np.nan      

### 5.2.3 Append `number_of_uninfected_contacts` and `number_of_effective_contacts`

In [100]:
data_matrix = np.hstack([data_matrix, number_of_uninfected_contacts.reshape(-1, 1)])
data_matrix = np.hstack([data_matrix, number_of_effective_contacts.reshape(-1, 1)])

In [ ]:
# Remove gender (because our model has no parameter that affect gender) and days of infection to recovered (because there were only 2 cases)
data_matrix = np.delete(data_matrix, (1, 3), 1)
# Remove number_of_uninfected_contacts because the data provided by Taiwan CDC is actually not number of uninfected contacts but size of the cluster.
data_matrix = np.delete(data_matrix, -2, axis=1)

print('number of non nan element in each column: ', np.sum(~np.isnan(data_matrix), axis=0))
# Check non-NaN counts
value_counts = np.sum(~np.isnan(data_matrix), axis=1)
plt.hist(value_counts, bins=range(max(value_counts)+2))
plt.xlabel('Number of non-NaN elements each subject in the Taiwan data matrix')
plt.ylabel('Frequency')
print(np.unique(value_counts, return_counts=True))

# plt.savefig('Taiwan_matrix_non_nan_RW2022.pdf')

In [ ]:
# Matrix size reduction, extract only the informative samples
data_matrix_small = data_matrix[value_counts >= 3, :]
print(data_matrix_small.shape)

if save_p_state:
    with open('./variable/Taiwan_data_matrix.npy', 'wb') as f:
        np.save(f, data_matrix_small)

    with open('./variable/Taiwan_data_matrix_full.npy', 'wb') as f:
        np.save(f, data_matrix)